# Лабораторная работа №5
## Задание
1. Произвести масштабирование признаков (scaling).
2. С использованием библиотеки scikit-learn написать программу с использованием алгоритмов кластеризации данных, позволяющую разделить исходную выборку на классы, соответствующие предложенной вариантом задаче.
3. Провести эксперименты и определить наилучший алгоритм кластеризации, параметры алгоритма. Необходимо использовать не менее 3-х алгоритмов. Данные экспериментов необходимо представить в отчете.

Данные: Mice Protein Expression.

## Выполнение работы
### Пункт 1
Импортируем нужные для работы библиотеки и прочитаем данные.

In [130]:
import numpy as np
import pandas as pd

from typing import Tuple
from numpy import random as rnd
from matplotlib import pyplot as plt
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import adjusted_rand_score
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression, Lasso, Ridge, Perceptron
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.cluster import KMeans, MeanShift, AgglomerativeClustering, SpectralClustering, FeatureAgglomeration
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA

target_feature = 'class'

metric = adjusted_rand_score

data = pd.read_excel('../../data/lab5.xls')

y = data[target_feature]
data.drop([target_feature], axis=1, inplace=True)
X = data

Сначала необходимо обработать данные. В сете имеются 3 столбца, которые по сути в сочетании с друг другом образуют целевой столбец, так что их уберем, ибо цель стоит выделить классы по протеинам. На одних только этих столбцах любой алгорит даст иделаьный результат, так что такая кластеризация не представляет интереса. Также имеется столбец `MouseID`, который не несет в себе никакой информации.

In [131]:
data.drop(['MouseID', 'Genotype', 'Treatment', 'Behavior'], axis=1, inplace=True)

Удалим пропуски.

In [132]:
X = data.copy()
X.dropna(axis=0, inplace=True)

y_init = y.copy()
y = y.iloc[X.index]

Теперь же произведем масштабирование признаков, но в этот раз будем использовать `MinMaxScaler`, который является функцией $f: X \to (0, 1)$. Выбор обоснован тем, что подобное масштабирование лучше разделяет признаки между собой, а учитывая, что классы, которые предстоит разделять, достаточно похожи, это является очень важным фактором.

In [133]:
scaler = MinMaxScaler()
X[list(X)] = scaler.fit_transform(X)

### Пункт 2
Воспользуемся алгоритмом `KMeans`, чтобы кластеризовать данные. Метрикой же будет `adjusted_rand_score`, ибо ее значение лежит в диапазоне $[-1, 1]$, что позволит проще определить насколько хороша или плоха модель.

In [134]:
def get_accuracy(X, y, model, **params):
    model = model.set_params(**params)
    preds = model.fit_predict(X)
    score = metric(y, preds)
    return score
    
    
def try_km(X, y, **args):
    base_params = {
        'n_clusters': 8,
        'verbose': 0,
        'random_state': 7,
        'n_jobs': -1
    }
    params = { **base_params, **args }

    model = KMeans()
    score = get_accuracy(X, y, model, **params)
    print('Kmeans score: {:.3f}'.format(score))
    
try_km(X, y)

Kmeans score: 0.202


Результат не является очень хорошим, поэтому есть смысл попробовать другие алгоритмы кластеризации, а также проанализировать данные тщательнее.

### Пункт 3
Для начала попробуем другие алгоритмы.

Проверим `GaussianMixture`, ибо обычно этот алгоритм лучше справляется с кучными данными.

In [135]:
def try_gm(X, y, **args):
    base_params = {
        'n_components': 8,
        'random_state': 7
    }
    params = { **base_params, **args }

    model = GaussianMixture()
    score = get_accuracy(X, y, model, **params)
    print('Gaussian mixture score: {:.3f}'.format(score))

try_gm(X, y)

Gaussian mixture score: 0.205


Не многим лучше, чем `KMeans`. Давайте теперь попробуем один из иерархальных алгоритмов - `AgglomerativeClustering`.

In [136]:
def try_ac(X, y, **args):
    base_params = {
        'n_clusters': 8
    }
    params = { **base_params, **args }
    
    model = AgglomerativeClustering()
    score = get_accuracy(X, y, model, **params)
    print('Agglomerative clustering score: {:.3f}'.format(score))

try_ac(X, y)

Agglomerative clustering score: 0.200


Аналогично. Все 3 алгоритма дают плюс минус один и тот же результат, так что скорее всего нам нужно лучше поработать с данными.

В части обработки данных мы удаляем все семплы, которые содержат нулевые значения, посмотрим сколько всего мы удалили.

In [137]:
print(f'Total samples deleted: {data.shape[0] - X.shape[0]} = {(data.shape[0] - X.shape[0]) / data.shape[0] * 100:.2f}%')

Total samples deleted: 528 = 48.89%


Получается, что мы удаляем практически половину данных, что точно не является лучшим решением. Давайте посмотрим, сколько пропусков содержится по колонкам.

In [138]:
cols = list(data)
print(len(cols))
for i in range(len(cols)):
    print(f'col #{i}, nans: {data[cols[i]].isna().sum() / data.shape[0] * 100:.2f}%')

77
col #0, nans: 0.28%
col #1, nans: 0.28%
col #2, nans: 0.28%
col #3, nans: 0.28%
col #4, nans: 0.28%
col #5, nans: 0.28%
col #6, nans: 0.28%
col #7, nans: 0.28%
col #8, nans: 0.28%
col #9, nans: 0.28%
col #10, nans: 0.28%
col #11, nans: 0.28%
col #12, nans: 0.28%
col #13, nans: 0.28%
col #14, nans: 0.28%
col #15, nans: 0.28%
col #16, nans: 0.28%
col #17, nans: 0.28%
col #18, nans: 0.28%
col #19, nans: 0.28%
col #20, nans: 0.28%
col #21, nans: 0.28%
col #22, nans: 0.28%
col #23, nans: 1.67%
col #24, nans: 0.28%
col #25, nans: 0.28%
col #26, nans: 0.28%
col #27, nans: 0.65%
col #28, nans: 0.28%
col #29, nans: 0.28%
col #30, nans: 0.28%
col #31, nans: 1.67%
col #32, nans: 0.28%
col #33, nans: 0.28%
col #34, nans: 0.28%
col #35, nans: 0.28%
col #36, nans: 0.28%
col #37, nans: 0.28%
col #38, nans: 0.28%
col #39, nans: 0.28%
col #40, nans: 0.28%
col #41, nans: 0.28%
col #42, nans: 0.28%
col #43, nans: 0.00%
col #44, nans: 0.00%
col #45, nans: 0.00%
col #46, nans: 0.00%
col #47, nans: 0.00%

Получается, что большая часть пропусков содержится в 68, 69, 73, 74 и 75 колонках, поэтому имеет смысл удалить эти колонки, а потом уже удалить семплы с пропусками.

In [139]:
X = data.copy()
print(data.shape)

many_nans = [68, 69, 73, 74, 75]
to_del = [list(X)[col_number] for col_number in many_nans]
X.drop(to_del, axis=1, inplace=True)
X.dropna(axis=0, inplace=True)

y = y_init.iloc[X.index]

print('Done!')

(1080, 77)
Done!


Посмотрим сколько теперь данных было удалено.

In [140]:
print(f'Total samples deleted: {data.shape[0] - X.shape[0]} = {(data.shape[0] - X.shape[0]) / data.shape[0] * 100:.2f}%')

Total samples deleted: 108 = 10.00%


Уже намного лучше. Следующим шагом будет замена `MinMaxScaler` на `StandardScaler` и применение `FeatureAgglomeration`. `FeatureAgglomeration` - уменьшает количество измерений путем группировки данных по схожести фич, на самом деле именно то, что нужно, ибо изначальная постановка задачи для нашим данных была именно такая.

In [141]:
data = X.copy()
scaler = MinMaxScaler()
X[list(X)] = scaler.fit_transform(X)

try_km(X, y)
try_gm(X, y)
try_ac(X, y)

Kmeans score: 0.158
Gaussian mixture score: 0.174
Agglomerative clustering score: 0.174


Ну что же, стало только хуже. Возможно это из-за того, что появилось много новых примеров, которые похожи друг на друга и разделить данные стало сложнее.

Следующим шагом попробуем уменьшить размерность данных: воспользуемся `FeatureAgglomeration` - работает также как и `AgglomerationClustering`, но для столблцов - объединяет их по схожести. Главным параметром модели является `n_clusters`, которое определяет результируещее количество фич.

Переберем параметры от 8 до 45, чтобы найти лучший. Также заменим `MinMaxScaler` на `StandardScaler`, ибо `FeatureAgglomeration` работает с ним лучше (согласно спецификации, да и на практике).

In [142]:
scaler = StandardScaler()
data[list(data)] = scaler.fit_transform(data)

for i in range(8, 46):
    print(f'n_clusters: {i}')
    data_copy = data.copy()
    model = FeatureAgglomeration()
    params = {
        'n_clusters': i
    }
    
    model = model.set_params(**params)
    data_copy = model.fit_transform(data_copy)
    y.reset_index(drop=True, inplace=True)
    data_copy = pd.DataFrame(data_copy)
    
    try_km(data_copy, y)
    try_gm(data_copy, y)
    try_ac(data_copy, y)
    print()

n_clusters: 8
Kmeans score: 0.151
Gaussian mixture score: 0.212
Agglomerative clustering score: 0.158

n_clusters: 9
Kmeans score: 0.153
Gaussian mixture score: 0.190
Agglomerative clustering score: 0.165

n_clusters: 10
Kmeans score: 0.160
Gaussian mixture score: 0.227
Agglomerative clustering score: 0.197

n_clusters: 11
Kmeans score: 0.151
Gaussian mixture score: 0.273
Agglomerative clustering score: 0.146

n_clusters: 12
Kmeans score: 0.152
Gaussian mixture score: 0.256
Agglomerative clustering score: 0.201

n_clusters: 13
Kmeans score: 0.140
Gaussian mixture score: 0.137
Agglomerative clustering score: 0.137

n_clusters: 14
Kmeans score: 0.127
Gaussian mixture score: 0.145
Agglomerative clustering score: 0.135

n_clusters: 15
Kmeans score: 0.139
Gaussian mixture score: 0.233
Agglomerative clustering score: 0.135

n_clusters: 16
Kmeans score: 0.147
Gaussian mixture score: 0.213
Agglomerative clustering score: 0.164

n_clusters: 17
Kmeans score: 0.134
Gaussian mixture score: 0.177
A

Больше внимания стоит уделить модели `GaussianMixture` потому что она дает самый лучший результат (0.273), а также у нее есть параметры, которые мы можем поменять, чтобы улучшить точность модели.
- `n_init` - количество инициализаций алгоритма, обычно ~20 хватает для оптимального значения;
- `covariance_type` - определяет тип ковариационной матрицы для компонент, попробуем значение **tied**.

In [143]:
for i in range(8, 46):
    print(f'n_clusters: {i}')
    data_copy = data.copy()
    model = FeatureAgglomeration()
    params = {
        'n_clusters': i
    }
    
    model = model.set_params(**params)
    data_copy = model.fit_transform(data_copy)
    y.reset_index(drop=True, inplace=True)
    data_copy = pd.DataFrame(data_copy)
    
    try_gm(data_copy, y, **{
        'n_init': 20,
        'covariance_type': 'tied'
    })
    print()

n_clusters: 8
Gaussian mixture score: 0.263

n_clusters: 9
Gaussian mixture score: 0.290

n_clusters: 10
Gaussian mixture score: 0.246

n_clusters: 11
Gaussian mixture score: 0.261

n_clusters: 12
Gaussian mixture score: 0.216

n_clusters: 13
Gaussian mixture score: 0.262

n_clusters: 14
Gaussian mixture score: 0.300

n_clusters: 15
Gaussian mixture score: 0.249

n_clusters: 16
Gaussian mixture score: 0.293

n_clusters: 17
Gaussian mixture score: 0.335

n_clusters: 18
Gaussian mixture score: 0.340

n_clusters: 19
Gaussian mixture score: 0.250

n_clusters: 20
Gaussian mixture score: 0.296

n_clusters: 21
Gaussian mixture score: 0.260

n_clusters: 22
Gaussian mixture score: 0.359

n_clusters: 23
Gaussian mixture score: 0.257

n_clusters: 24
Gaussian mixture score: 0.251

n_clusters: 25
Gaussian mixture score: 0.390

n_clusters: 26
Gaussian mixture score: 0.260

n_clusters: 27
Gaussian mixture score: 0.331

n_clusters: 28
Gaussian mixture score: 0.298

n_clusters: 29
Gaussian mixture scor

Как мы видим, изменение параметров очень сильно улучшило качество модели (0.273 -> 0.460), что уже является приемлемым качеством, учитывая сложность задачи.

## Вывод
В ходе выполнения лабораторной работы были разобраны:
- 3 различные модели кластеризации (`Kmeans`, `GaussianMixture`, `AgglomerativeClustering`);
- способ уменьшения размерности/избыточности данных - `FeatureAgglomeration`;
- тюнинг параметров для улучшения точности модели.
В итоге удалось поднять изначальную лучшую точность среди трех моделей 0.205 до 0.460, что является более чем 2-ух кратным увеличением.